# Text generation with an RNN

Adapted from the [tensorflow example](https://www.tensorflow.org/tutorials/sequences/text_generation), to run on [datahub.ucsd.edu](datahub.ucsd.edu), with TF 1.14. 

Robert Twomey, rtwomey@ucsd.edu.

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly to seuquentially add characters.

The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Goals

- Understand the evolution of samples during training, and the relationship between the number of training ephocs and the quality of the generated output. 
- Produce (or download) your own textual corpus to experiment with parameters, training, and results. In [his article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) cited above, Karpathy plays with sources including Shakespeare, Wikipedia articles, Algebraic Geometry (Latex), and Linux source code, and baby names.
- Experiment with:
  - The effect of the number of training epochs
  - The parameters, training, and results. 
  - Add additional RNN layers
- Share your results with class.

## Background

"Neural Nets" are loosely inspired on neurobiology. Now people call them units, nodes, etc., as opposed to "neurons". Each unit has a number of inputs (with weights), an activation function, a bias (offset), and an output. These weights and biases are the trainable parameters of the network that can be "learned".

![A_neuron_a_simplified_mathematical_model](images/neuronal_model.png)

Neural Networks are interconnected layers of these kinds of neuronal units: 

<!--![Layered_net](images/layers.png)-->
<img src="images/layers.png" width=400px>

Each unit has an activation function, a bias, and ever connection going in or out of the node has a weight. All of these are trainable parameters of the network. For the image above, we have 9 neurons, with 32 connections (each with weight), 9 biases. This makes for 41 trainable parameters). This diagram is a fully connected network (each node connects to every node on the next layer). We call this a feed-forward network, because input data (at the left) is propogated / "feeds forward" from node to node in layer to layer, producing outputs (at the right).

Complicated feed forward mechanisms (like this MNIST handwritten digit classifier, below) might have 13k trainable parameters. These are the parameters (weights and biases) of the network that are learned during training. The RNN model we use below has 4M trainable parameters.

<!--![Mnist](images/mnist.png)-->
<img src="images/mnist.png" width=400px>

Today we are working with Recurrent Neural Networks (RNNs). They are a special kind of network where the system feed data back into itself (see below). We evaluate them sequentially: computing values, then passing information forward to the network to evaluate the next item in a sequence. They are very prevalent for modeling sequences of data, whether text, or time series like drawings or music. 

<!--![RNN Unit](images/rnn_unit.png)-->
<img src="images/rnn_unrolled.png" width=600px> 

## Setup

### Import TensorFlow and other libraries

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image


### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
# Default example training on Shakespeare:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Use your own file:
path_to_file = "myNovel.txt"

# Robert's file
# path_to_file = "script.txt"




### Read the data

First, look in the text.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 4830667 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Book 1 Chapter 1Previous Chapter
B1C1: Qin Yu
It is a day in the winter. After heavy snows, the whole Yan City seems to be covered in a silvery coat. A very large city, Yan City can accommodate several million people, and so the mansion of the person


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

160 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Book 1 Chapte' ---- characters mapped to int ---- > [30 72 72 68  1 15  1 31 65 58 73 77 62]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. 

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
 print(idx2char[i.numpy()])

B
o
o
k
 


The `batch` method lets us easily convert these individual characters to sequences of the desired size. 

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
 print(repr(''.join(idx2char[item.numpy()])))

'Book 1 Chapter 1Previous Chapter\nB1C1: Qin Yu\nIt is a day in the winter. After heavy snows, the whole Yan City seems to be covered in a silvery coat. A very large city, Yan City can accommodate several'
' million people, and so the mansion of the person who is in charge of the 3 counties comprising the Eastern region -- â\x80\x98East Vanquishing Princeâ\x80\x99 Qin De -- is located in the city.\nThe East Vanquishin'
'g Prince mansion occupies an extremely large area with its main entrance kept open all the time. The main entrance is also very large, enough for 6 or 7 people to enter at the same time.\nMoreover, two '
'bare-chested vigorous men who are about 2 meters tall and look as if they are carved from rock are standing on the left and right hand sides of the entrance with their eyes coldly glancing at the crowd'
's of passersby. Each of them is carrying a huge blood-red battle knife on his muscular tiger-like back. At first sight, the knives are at least 1.5 m long.\nIt is a wi

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Book 1 Chapter 1Previous Chapter\nB1C1: Qin Yu\nIt is a day in the winter. After heavy snows, the whole Yan City seems to be covered in a silvery coat. A very large city, Yan City can accommodate severa'
Target data: 'ook 1 Chapter 1Previous Chapter\nB1C1: Qin Yu\nIt is a day in the winter. After heavy snows, the whole Yan City seems to be covered in a silvery coat. A very large city, Yan City can accommodate several'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('B')
  expected output: 72 ('o')
Step    1
  input: 72 ('o')
  expected output: 72 ('o')
Step    2
  input: 72 ('o')
  expected output: 68 ('k')
Step    3
  input: 68 ('k')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 15 ('1')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [14]:
# Batch size (number of 100 word sequences to train on)
BATCH_SIZE = 64

# How many individual training steps per samples (steps)
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

### Neuronal Modal


__NOTE__: Insert an image here diagramming basic neural net

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

Use `CuDNNGRU` if running on GPU.  

In [16]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),

    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 160) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length: 

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           40960     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 160)           164000    
Total params: 4,143,264
Trainable params: 4,143,264
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. 

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [21]:
# sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) # TF 1.12
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [22]:
sampled_indices

array([129,  59,  66,   9,  41,  19,  48,  76,  39, 124, 101,  15,  15,
        22,   4, 137,  24,  90, 138,  30,  79,  40,   0,  19,  12,  13,
       129, 101,  64,  20, 125,  94,  24,  21,  43, 157,  27, 111, 128,
       159,  80, 152,  12, 150,  23, 140, 141,  71,  57,   8, 112,  46,
        19,  19,   3,  83,   0,  82,   9, 110,  90,  65,  78,  12,  60,
        56,  30,   3,  84, 127, 114,  89,  83,  66,  51,  77,   0,  97,
        63,  80,  53, 123,  81,  64, 140, 121, 100,  22,  69,  95,  51,
       154, 136,  30,  92,  27, 139,  77,  80,  63, 105,  38,  88,  35,
        37,  53, 141,  18,  68,  11,  93,  45, 134, 141, 128, 109,  53,
       158,  29,  80,  48,   3, 105,  91, 121,  99, 158, 132, 137,  79,
       130,  52,  92,  54,  14, 127, 120,  86, 135, 110, 116,  69,  29,
       129,  85, 159,  88,  70,  40, 151,  52, 113, 126,  52,  94,  56,
       135,  63, 157,  60,  83, 125,  87,  27, 135, 126,  34,  93,  42,
       151, 117, 140,  96,   5,  39, 101,  84,  83,  15,   7, 10

Decode these to see the text predicted by this untrained model:

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ld turtle Qingxuan in the Qingxuan Ridge,\nOld Founder Qingxuan is sitting in an armchair. There is a lady-in-waiting gently fanning him with a fan on either side of him. He is all smiles at the moment'

Next Char Predictions: 
 '¬bi*M5TsK§\x90118#´:\x85µBvL\n5./¬\x90g6¨\x89:7Oè>\x9a«ïwã.Ã9·¸n_)\x9bR55"z\ny*\x99\x85hu.c]B"~ª\x9d\x84ziWt\n\x8cfwY¦xg·¤\x8f8l\x8aWå³B\x87>¶twf\x94J\x83GIY¸4k-\x88Q±¸«\x98YéAwT"\x94\x86¤\x8eé¯´v\xadX\x87Z0ª£\x81²\x99\x9flA¬\x80ï\x83mLâX\x9c©X\x89]²fècz¨\x82>²©F\x88Nâ\xa0·\x8b%K\x90~z1(\x973=N\x9bEb~s\xa0\x92TflCh®:%\x86'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions. 

Because our model returns logits, we need to set the `from_logits` flag. 


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 160)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.0750704


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function.

NOTE TO SELF: Insert reference about optimizers, gradient descent, and Adam particularly.

In [25]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. Increase the number of epochs to increase training time.

In [27]:
EPOCHS=20

In [28]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/20
377/377 [==============================] - 32s 84ms/step - loss: 2.3201
Epoch 2/20
377/377 [==============================] - 29s 77ms/step - loss: 1.3479
Epoch 3/20
377/377 [==============================] - 29s 78ms/step - loss: 1.1080
Epoch 4/20
377/377 [==============================] - 29s 78ms/step - loss: 1.0199
Epoch 5/20
377/377 [==============================] - 30s 78ms/step - loss: 0.9695
Epoch 6/20
377/377 [==============================] - 30s 79ms/step - loss: 0.9356
Epoch 7/20
377/377 [==============================] - 29s 78ms/step - loss: 0.9096
Epoch 8/20
377/377 [==============================] - 29s 78ms/step - loss: 0.8879
Epoch 9/20
377/377 [==============================] - 29s 78ms/step - loss: 0.8707
Epoch 10/20
377/377 [==============================] - 29s 77ms/step - loss: 0.8548
Epoch 11/20
377/377 [==============================] - 29s 77ms/step - loss: 0.8410
Epoch 12/20
377/377 [==============================] - 29s 76ms/step - loss: 0.8285
E

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training/ckpt_20'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            40960     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 160)            164000    
Total params: 4,143,264
Trainable params: 4,143,264
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 10000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.3

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"Qin"))


W1021 04:17:46.856163 139922928318272 deprecation.py:323] From <ipython-input-32-f7f1edbfabea>:29: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


Qingxï¼¦
It is not a single piece of over 100 m long and a country fast and a great number of disciples were seen everywhere in the capital. If you want to get into action, Iâll tell you a chance to obtain the Ring of the Blood-red Cave. Iâll also give a suitable place as well.â Qin Yu says with a smile.
The 3 Golden Flame eagle is standing on the surface of the secret room. The 2 of them then leave the water directly in a strike and a sweep of the seniors in his hand then disappears in a corner of the main hall.
The Flaming Sword is a secret book in his eyes.
âThis is the largest practice method of the Qin clan. I want to see who was about to meet his fatherâs level of power. The one who is carrying in a secret book, as if he has been such a short time, he intends to another would not be able to rely on his speed.
âWhat do you want to let the other guys like this?â Qin Yu says with a smile.
The 3 brothers, Qin Yu and Liâer are still attacked by the 3 majesties simply b

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

# Activities

1. Gradually increase the number of training epochs, and see how your generated output evolves. "Sample" the model a number of times and store your results.
2. Train on your own corpus, instead of Shakespeare.
  - Find a plain text data source and upload it to datahub.
  - How long does your text need to be? (shakespeare is 1.1M characters)
3. Experiment with the network parameters. In ML we call these "hyperparameters", parameters of the network, to distinguish from the internal weights and biases ("parameters") that are learned during training.
  - How does changing ```temperature = 1.0``` change the qualities of the generated text? E.g. try values 0 < 1.0, > 1.0, near 0.0
  - How does changing the number of embedding dimensions ```embedding_dim = 256``` change the results? This is more important in word-level models.
  - How does changing the number of RNN units (internal "nodes" in the LSTM) change the results? By default we have ```rnn_units = 1024```
  - Does changing the length of training sequences ```seq_length = 100``` change the results? 
  - Add a new layer to your network (copy the rnn statement in the model construction)
4. Future directions: 
  - Move on to a word-level RNN model (like this https://github.com/hunkim/word-rnn-tensorflow)
  - Move on to fine-tuning the GPT-2 model (https://github.com/nshepperd/gpt-2, https://pmbaumgartner.github.io/blog/gpt2-jokes/)